In [1]:
import urllib
from urllib import request
from bs4 import BeautifulSoup
import os
import requests
import argparse
import re
import time
import json
import math
import codecs
import threading
import urllib.parse as up

In [2]:
url = r'https://www.semanticscholar.org/search?fos%5B0%5D=computer-science&q=transformer&sort=relevance&page=6'
html = str(request.urlopen(url).read(), 'utf-8')
soup = BeautifulSoup(html, 'lxml')

In [3]:
'asdf'+'etr'+'gd'

'asdfetrgd'

In [4]:
def SSSQuery(query, num_item=50, offset=0):
    query = '+'.join(query.split())
    url = f'https://api.semanticscholar.org/graph/v1/paper/search?query={query}&offset={offset}&limit={num_item}'
    paper_list = str(request.urlopen(url).read(), 'utf-8')
    paper_list = json.loads(paper_list)

    tldr_cat = ''
    for paper in paper_list['data']:
        paper_id = paper['paperId']
        paper_url = f'https://api.semanticscholar.org/graph/v1/paper/{paper_id}?fields=citations.authors,tldr,'
        paper_info = str(request.urlopen(paper_url).read(), 'utf-8')
        paper_info = json.loads(paper_info)
        tldr = paper_info['tldr']
        if tldr is not None:
            tldr_cat += ' '+tldr['text']
        
        

In [5]:
num_item = 30
offset = 0
query = 'transformer test'
query = '+'.join(query.split())
url = f'https://api.semanticscholar.org/graph/v1/paper/search?query={query}&offset={offset}&limit={num_item}'
paper_list = str(request.urlopen(url).read(), 'utf-8')
paper_list = json.loads(paper_list)
print(paper_list)

{'total': 12364998, 'offset': 0, 'next': 30, 'data': [{'paperId': '32186d6d0e11039bd4c0aca43b3c7858b2fe0ed0', 'title': 'Calibration of Instrument Current Transformer Test Sets'}, {'paperId': 'c0ab29fec845d7e7bcedf50cc05bad440dfd02cd', 'title': 'A Computer-Controlled Calibrator for Instrument Transformer Test Sets'}, {'paperId': '2c38ce741b2d14510348450225a6cf27fd11d79a', 'title': 'Analysis of Quality Problems in Power Transformer Test Check and Their Control Measure Research'}, {'paperId': '0170fc76e934ee643f869df18fb617d5357e8b4e', 'title': 'Conformer: Convolution-augmented Transformer for Speech Recognition'}, {'paperId': 'f481daf0a28292ac36e94611e538c1a3b3ad71c5', 'title': 'Non-conventional instrument current transformer test set for industrial applications'}, {'paperId': '8d6f2fb79644f0664b6c2944e00a358802ac18cb', 'title': 'Instrument Transformer Test Set calibration using digital sampling'}, {'paperId': 'f2e655902bc78fe3f796cb40b03f7fe030d79beb', 'title': 'A current transformer te

In [6]:
for paper in paper_list['data']:
    paper_id = paper['paperId']
    paper_url = f'https://api.semanticscholar.org/graph/v1/paper/{paper_id}?fields=citations.authors,tldr'
    paper_info = str(request.urlopen(paper_url).read(), 'utf-8')
    paper_info = json.loads(paper_info)
    try:
        print(paper_info['tldr']['text'])
    except:
        pass

The paper presents the basic layout for instrument current transformer calibration using comparative method with a standard and results obtained using described procedure are presented.
A new calibrator is described that reaches uncertainties of the order of $10\times 10^{-6}$ while being suitable for the calibration of test sets for nonconventional analog instrument transformers and fully computer-controlled.
This work proposes the convolution-augmented transformer for speech recognition, named Conformer, which significantly outperforms the previous Transformer and CNN based models achieving state-of-the-art accuracies.
A test set for the calibration of non-conventional current instrument transformer working with current down to the 0.1 mA level, intended for use in industrial environment and with target uncertainties within 100 μA/A and 100 μrad.
The architecture improves both the image encoding and the language generation steps: it learns a multi-level representation of the relation

In [ ]:
def log(*args, end=None):
    if end is None:
        print(time.strftime("==> [%Y-%m-%d %H:%M:%S]",
                            time.localtime()) + " " + "".join([str(s) for s in args]))
    else:
        print(time.strftime("==> [%Y-%m-%d %H:%M:%S]", time.localtime()) + " " + "".join([str(s) for s in args]),
              end=end)

def SSSCrawl(urls, outpath):
    for url in urls:
        tldr_name = up.unquote(url).split('/')[-1].replace(' ', '-')[15:]
        tldr_path = os.path.join(outpath, 'txt', tldr_name)
        try:
            tldr = requests.get(url)
            with open(tldr_path, 'wb') as f:
                f.write(tldr.content)
                f.flush()
            log("Successfully crawled tldr:{}".format(tldr_name[:60]))
        except:
            log("Failed crawling tldr:{}".format(url))
            
def SSSCrawlMain(url, outpath):
    if not os.path.exists(outpath):
        os.mkdir(outpath)
    if not os.path.exists(os.path.join(outpath, 'txt')):
        os.mkdir(os.path.join(outpath, 'txt'))
    html = str(request.urlopen(url).read(), 'utf-8')
    soup = BeautifulSoup(html, 'lxml')
    try:
        pagesum = int(soup.body.select(
            '#content #post-list .content #paginator .pagination a')[-2].get_text())
    except IndexError:
        pagesum = 1

    log('Start crawling tldr links...')
    links = []
    for pagenum in range(1, pagesum+1):
        if page>1:
            _url = url + '&page=' + str(pagenum)
        html = str(request.urlopen(_url).read(), 'utf-8')
        soup = BeautifulSoup(html, 'lxml')
        links.extend([i.attrs['href'] for i in soup.body.select(
            '#content > #post-list > .content > div > #post-list-posts > li > a')])
    log('Finished crawling image links.')

    with codecs.open(os.path.join(outpath, 'url_list.txt'), 'w+') as f:
        for link in links:
            f.write(link+'\n')

    def chunks(arr, m):
        n = int(math.floor(len(arr) / float(m)))
        return [*[arr[i:i + n] for i in range(0, (m-1)*n, n)],arr[(m-1)*n:]]

    threads = []
    links_sep = chunks(links, opt.thread_num)

    for i in range(opt.thread_num):
        threads.append(imgThread(i, links_sep[i], outpath))

    for i in threads:
        i.start()

    for i in threads:
        i.join()
    log('Successfully crawled all of the images. Thanks for using.')

In [5]:
pip install lxml


Note: you may need to restart the kernel to use updated packages.


In [ ]:
if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument('--key', default='Visible Light Communication', help='The name of tag you want to crawl.')
    parser.add_argument('--thread_num', type=int, default=5, help='The number of threads you want to use when crawling.')
    opt = parser.parse_args()
    KonaTagCrawlMain('https://www.semanticscholar.org/search?q='+opt.key+'sort=relevance', opt.key)